### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items[:500000].groupby(['user_id', 'movie_id'])['rating'].max().unstack()

In [3]:
# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[ 7.  7. 10.  8.]
 [10.  8. 10. 10.]
 [ 5.  8.  5.  8.]
 [10.  8.  1. 10.]
 [ 9. 10. 10.  9.]
 [10.  4.  9. 10.]
 [ 9.  8. 10.  9.]
 [ 6.  4. 10.  6.]
 [10.  9. 10.  8.]
 [10. 10.  9.  9.]
 [ 8.  7. 10.  7.]
 [ 9.  9. 10. 10.]
 [ 9.  5.  9.  7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the funtion on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [4]:
ratings_mat.shape

(13, 4)

In [9]:
np.count_nonzero(~np.isnan(ratings_mat))

52

In [12]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]# number of rows in the matrix
    n_movies = ratings_mat.shape[1] # number of movies in the matrix
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat)) # total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users, latent_features)
    movie_mat = np.random.rand(latent_features, n_movies)
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # keep track of iteration and MSE
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):
        
        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                
                # if the rating exists
                if ratings_mat[i,j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])

                    # Keep track of the total sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*diff*user_mat[i, k])

        # print results
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting **U** and **V** matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [13]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=10)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 46.659085
2 		 22.151264
3 		 7.219885
4 		 3.450801
5 		 2.922594
6 		 2.820733
7 		 2.771912
8 		 2.732013
9 		 2.691528
10 		 2.647216


In [14]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 8.25717359  7.44771262  8.85907843  7.94077805]
 [10.13332603  8.28215951 10.29120785  9.52857988]
 [ 6.73195202  5.91025236  6.70091251  6.77642793]
 [ 7.25457962  6.61223676  7.33865661  7.07685856]
 [ 9.59745437  8.88998958  9.66093097  9.60222482]
 [ 8.81348009  7.14538508  8.77833817  8.93013563]
 [ 9.22127565  8.04428406  9.609241    9.15813398]
 [ 7.06641784  5.74516348  7.33998044  6.5174262 ]
 [ 9.51287162  8.18359356  9.86405377  8.91643367]
 [ 9.56537179  8.45371755  9.87484041  9.49280727]
 [ 8.15450754  7.59264094  8.50674915  7.88255698]
 [ 9.87558247  8.46566887 10.1269777   9.66836852]
 [ 8.19481908  6.30135277  8.21231224  7.40650877]]
[[ 7.  7. 10.  8.]
 [10.  8. 10. 10.]
 [ 5.  8.  5.  8.]
 [10.  8.  1. 10.]
 [ 9. 10. 10.  9.]
 [10.  4.  9. 10.]
 [ 9.  8. 10.  9.]
 [ 6.  4. 10.  6.]
 [10.  9. 10.  8.]
 [10. 10.  9.  9.]
 [ 8.  7. 10.  7.]
 [ 9.  9. 10. 10.]
 [ 9.  5.  9.  7.]]


**Write your summary here.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting **U** and **V** matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [15]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 42.289350
2 		 17.862130
3 		 5.802028
4 		 3.221838
5 		 2.852099
6 		 2.742011
7 		 2.660560
8 		 2.580879
9 		 2.496688
10 		 2.405569
11 		 2.306584
12 		 2.199677
13 		 2.085515
14 		 1.965439
15 		 1.841420
16 		 1.715945
17 		 1.591825
18 		 1.471941
19 		 1.358938
20 		 1.254957
21 		 1.161431
22 		 1.078999
23 		 1.007539
24 		 0.946309
25 		 0.894132
26 		 0.849604
27 		 0.811268
28 		 0.777743
29 		 0.747807
30 		 0.720428
31 		 0.694772
32 		 0.670189
33 		 0.646184
34 		 0.622398
35 		 0.598581
36 		 0.574568
37 		 0.550265
38 		 0.525637
39 		 0.500694
40 		 0.475484
41 		 0.450088
42 		 0.424611
43 		 0.399175
44 		 0.373919
45 		 0.348989
46 		 0.324532
47 		 0.300695
48 		 0.277613
49 		 0.255413
50 		 0.234203
51 		 0.214073
52 		 0.195091
53 		 0.177305
54 		 0.160741
55 		 0.145402
56 		 0.131276
57 		 0.118332
58 		 0.106526
59 		 0.095803
60 		 0.086101
61 		 0.077351
62 		 0.069484
63 		 0.062426
64 		

In [16]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 7.00008126  7.00002787 10.00001018  7.99992172]
 [10.00029499  8.00010012 10.00003517  9.99971499]
 [ 5.0005173   8.00016679  5.00005728  7.9995055 ]
 [10.00059692  8.00019094  1.00006348  9.9994252 ]
 [ 9.0003351  10.00009819 10.00002667  8.9996714 ]
 [ 9.99960699  3.99985891  8.99994152 10.00036454]
 [ 8.99894547  7.9996515   9.9998709   9.00099166]
 [ 6.00149268  4.0004896  10.00017634  5.99858634]
 [10.00113796  9.00036149 10.00012133  7.99891048]
 [ 9.99929228  9.99975113  8.99989649  9.00065119]
 [ 7.99947426  6.99981913  9.9999287   7.00048991]
 [ 8.99929836  8.99976758  9.99991302 10.0006579 ]
 [ 9.00009714  5.00003588  9.00001364  6.99990544]]
[[ 7.  7. 10.  8.]
 [10.  8. 10. 10.]
 [ 5.  8.  5.  8.]
 [10.  8.  1. 10.]
 [ 9. 10. 10.  9.]
 [10.  4.  9. 10.]
 [ 9.  8. 10.  9.]
 [ 6.  4. 10.  6.]
 [10.  9. 10.  8.]
 [10. 10.  9.  9.]
 [ 8.  7. 10.  7.]
 [ 9.  9. 10. 10.]
 [ 9.  5.  9.  7.]]


**Write your summary here.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [17]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[nan,  7., 10.,  8.],
        [10.,  8., 10., 10.],
        [ 5.,  8.,  5.,  8.],
        [10.,  8.,  1., 10.],
        [ 9., 10., 10.,  9.],
        [10.,  4.,  9., 10.],
        [ 9.,  8., 10.,  9.],
        [ 6.,  4., 10.,  6.],
        [10.,  9., 10.,  8.],
        [10., 10.,  9.,  9.],
        [ 8.,  7., 10.,  7.],
        [ 9.,  9., 10., 10.],
        [ 9.,  5.,  9.,  7.]])

In [18]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 48.407336
2 		 24.559505
3 		 8.462366
4 		 3.652489
5 		 2.803394
6 		 2.601792
7 		 2.486250
8 		 2.380715
9 		 2.271058
10 		 2.153645
11 		 2.027841
12 		 1.894421
13 		 1.755114
14 		 1.612379
15 		 1.469189
16 		 1.328749
17 		 1.194174
18 		 1.068164
19 		 0.952736
20 		 0.849079
21 		 0.757540
22 		 0.677737
23 		 0.608751
24 		 0.549343
25 		 0.498163
26 		 0.453896
27 		 0.415361
28 		 0.381560
29 		 0.351678
30 		 0.325072
31 		 0.301238
32 		 0.279783
33 		 0.260399
34 		 0.242834
35 		 0.226880
36 		 0.212359
37 		 0.199114
38 		 0.187007
39 		 0.175911
40 		 0.165712
41 		 0.156306
42 		 0.147600
43 		 0.139512
44 		 0.131969
45 		 0.124905
46 		 0.118266
47 		 0.112003
48 		 0.106076
49 		 0.100450
50 		 0.095097
51 		 0.089993
52 		 0.085119
53 		 0.080459
54 		 0.076000
55 		 0.071731
56 		 0.067644
57 		 0.063733
58 		 0.059992
59 		 0.056415
60 		 0.053000
61 		 0.049741
62 		 0.046637
63 		 0.043683
64 		

In [19]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 7.550729578443146:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [20]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=4, learning_rate=0.005, iters=20)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 19.488164
2 		 9.044293
3 		 6.308620
4 		 4.931654
5 		 4.078802
6 		 3.489259
7 		 3.052380
8 		 2.713135
9 		 2.440760
10 		 2.216629
11 		 2.028861
12 		 1.869554
13 		 1.733193
14 		 1.615729
15 		 1.514043
16 		 1.425637
17 		 1.348458
18 		 1.280794
19 		 1.221204
20 		 1.168477


`6.` Now that you have a set of predictions for each user-movie pair.  Let's answer a few questions about your results. Provide the correct values to each of the variables below, and check your solutions using the tests below.

In [21]:
# Replace each of the comments below with the correct values
num_ratings = np.count_nonzero(~np.isnan(first_1000_users)) # How many actual ratings exist in first_1000_users
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()


ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings # How many ratings did we make for user-movie pairs that didn't actually have ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 14771.

The number of ratings made for user-movie pairs that didn't have ratings is 27902229


In [ ]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")